In [1]:
import matplotlib.pyplot as plt
import pandas
import numpy as np
import pycaret.classification as pc

%matplotlib inline
model_name = 'wine_model'

from pycaret.utils import version
version()

'2.3.6'

# Vinhos


## Leitura da Base 

In [2]:
df_wine = pandas.read_csv('../Data/dataset_vinhos.csv',sep=';')
wine_target_col = 'target'
wine_label_map = df_wine[['target', 'target_label']].drop_duplicates()
print(df_wine.shape)
df_wine.head()

(5320, 14)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,target,target_label,type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0,baixa qualidade,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0,baixa qualidade,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0,baixa qualidade,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1,alta qualidade,red
4,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,0,baixa qualidade,red


## Configuração do PyCaret 

In [ ]:
# pc.setup?

In [3]:
experiment = pc.setup(session_id=123,
                      data = df_wine, # Configurações de dados
                      train_size=0.6,
                      target = wine_target_col,
                      profile = False, # Analise interativa de variaveis
                      fold_strategy = 'stratifiedkfold', # Validação cruzada
                      fold = 10,
                      normalize = True,  # Normalização, transformação e remoção de variáveis
                      transformation = True, 
                      ignore_low_variance = True,
                      remove_multicollinearity = True,
                      multicollinearity_threshold = 0.95,
                      bin_numeric_features = None, # Binarizacao de variaveis
                      group_features = None, # Grupos de variáveis para combinar na engenharia de variaveis
                      categorical_features = ['type'],
                      ignore_features = ['target_label'],
                      log_experiment = True, # Logging dos experimentos e afins
                      experiment_name = 'wine-model',
                      silent=True)

,Description,Value
0,session_id,123
1,Target,target
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(5320, 14)"
5,Missing Values,False
6,Numeric Features,11
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


## Seleção de Modelos

In [ ]:
# pc.models()
# pc.compare_models?

In [ ]:
# pc.models()

In [ ]:
# O PyCaret treina os modelos com hiperparametros defaults, sem ajuste fino.
model = pc.compare_models(n_select = 1, sort='Accuracy', include=['lr', 'dt', 'rf'])

IntProgress(value=0, description='Processing: ', max=19)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,20:09:39
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Classifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)


Exception in thread Thread-10:
Traceback (most recent call last):
  File "C:\Users\BZ241WX\AppData\Local\Continuum\anaconda3\envs\infnet-ead\lib\site-packages\psutil\_pswindows.py", line 679, in wrapper
    return fun(self, *args, **kwargs)
  File "C:\Users\BZ241WX\AppData\Local\Continuum\anaconda3\envs\infnet-ead\lib\site-packages\psutil\_pswindows.py", line 863, in kill
    return cext.proc_kill(self.pid)
PermissionError: [WinError 5] Access is denied: '(originated from OpenProcess)'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\BZ241WX\AppData\Local\Continuum\anaconda3\envs\infnet-ead\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\BZ241WX\AppData\Local\Continuum\anaconda3\envs\infnet-ead\lib\site-packages\joblib\externals\loky\process_executor.py", line 558, in run
    self.terminate_broken(bpe)
  File "C:\Users\BZ241WX\AppData\Local\Continuum\anaconda3\envs\infnet-ead\lib\s

In [ ]:
model

## Ajuste de Hiperparâmetros 

In [ ]:
# pc.tune_model?

In [ ]:
tuned_model = pc.tune_model(model,
                            optimize = 'f1',
                            search_library = 'scikit-learn',
                            search_algorithm = 'random',
                            n_iter = 4)


## Avaliação do Modelo 

In [ ]:
# pc.evaluate_model?
# pc.plot_model?

In [ ]:
pc.evaluate_model(tuned_model)

## Interpretação dos Resultados 

In [ ]:
# pc.interpret_model?

In [ ]:
pc.interpret_model(tuned_model, plot='correlation', feature='alcohol')

## Calibração de Modelos 

In [ ]:
# pc.calibrate_model?

In [ ]:
calibrated_model = pc.calibrate_model(tuned_model, method='sigmoid', calibrate_fold=5, fold=5)
pc.plot_model(calibrated_model, plot='calibration')

## Optimização da Operação 

In [ ]:
# pc.optimize_threshold(calibrated_model, optimize = 'Precision');

## Avaliação do Teste 

In [ ]:
# pc.predict_model?

In [ ]:
pred_holdout = pc.predict_model(calibrated_model) # holdout set (test)

In [ ]:
pred_holdout.head()

In [ ]:
pred_holdout.Score.sum()
print(metrics.classification_report())

## Finalização do Modelo para Produção

In [ ]:
# pc.finalize_model?

In [ ]:
final_model = pc.finalize_model(calibrated_model)
pred_final = pc.predict_model(final_model) # holdout set (test)

## Exportação do Experimento 

In [ ]:
# pc.load_config?
# pc.save_config?
# pc.load_model?
# pc.save_model?

In [ ]:
pc.save_model(calibrated_model, model_name)